In [1]:
pwd()

"/Users/nmayhall/code/FermiCG-data/cr-qubit"

In [2]:
using Pkg; Pkg.activate("../../FermiCG/")
using FermiCG, NPZ
readdir()

  Activating project at `~/code/FermiCG`
┌ Info: Precompiling FermiCG [855c3b3a-d9bb-4919-8b00-0b3cfbdffb6f]
└ @ Base loading.jl:1423


11-element Vector{String}:
 ".ipynb_checkpoints"
 "C_act_loc_sorted_3.molden"
 "cr_qubit_cmf_bst1.ipynb"
 "cr_qubit_cmf_bst2.ipynb"
 "cr_qubit_scf.ipynb"
 "cr_qubit_scf_integrals_h0.npz.npy"
 "cr_qubit_scf_integrals_h1.npz.npy"
 "cr_qubit_scf_integrals_h2.npz.npy"
 "cr_qubit_scf_mo_coeffs.npz.npy"
 "excitations1.xlsx"
 "~\$excitations1.xlsx"

In [3]:
# load integrals from disk
h0 = npzread("cr_qubit_scf_integrals_h0.npz.npy")
h1 = npzread("cr_qubit_scf_integrals_h1.npz.npy")
h2 = npzread("cr_qubit_scf_integrals_h2.npz.npy")
ints = InCoreInts(h0, h1, h2)

ints_original = deepcopy(ints);

print(" Integrals have the following sizes: h0= ", size(h0), " h1= ", size(h1), " h2= ", size(h2))


 Integrals have the following sizes: h0= () h1= (34, 34) h2= (34, 34, 34, 34)

In [4]:
# Define clusters - this should probably be done in the python notebook, and then just read in here
clusters_in = [
    (1:10),   # metal
    (11:16), # Benzene 1
    (17:22), # Benzene 1
    (23:28), # Benzene 1
    (29:34), # Benzene 1
]

clusters = [Cluster(i,collect(clusters_in[i])) for i = 1:length(clusters_in)]

init_fspace = [
    (5,5),
    (3,3),
    (3,3),
    (3,3),
    (3,3)];
display(clusters)
display(init_fspace)

5-element Vector{Cluster}:
 Cluster(1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
 Cluster(2, [11, 12, 13, 14, 15, 16])
 Cluster(3, [17, 18, 19, 20, 21, 22])
 Cluster(4, [23, 24, 25, 26, 27, 28])
 Cluster(5, [29, 30, 31, 32, 33, 34])

5-element Vector{Tuple{Int64, Int64}}:
 (5, 5)
 (3, 3)
 (3, 3)
 (3, 3)
 (3, 3)

In [5]:
rdm1 = zeros(size(ints.h1))

e_cmf, U, da1, db1 = FermiCG.cmf_oo(ints, clusters, init_fspace, rdm1, rdm1, verbose=0, gconv=1e-6, method="cg", sequential=true);

 ooCMF Iter:    1 Total= -2119.151029564707 Active= -105.878811926138 G=     3.20e-01
 ooCMF Iter:    2 Total= -2119.184761796472 Active= -105.912544157904 G=     8.43e-02
 ooCMF Iter:    3 Total= -2119.186822074874 Active= -105.914604436305 G=     4.04e-02
 ooCMF Iter:    4 Total= -2119.187368537576 Active= -105.915150899007 G=     2.52e-02
 ooCMF Iter:    5 Total= -2119.187619893115 Active= -105.915402254546 G=     2.27e-02
 ooCMF Iter:    6 Total= -2119.187873158218 Active= -105.915655519650 G=     1.52e-02
 ooCMF Iter:    7 Total= -2119.187939366971 Active= -105.915721728403 G=     5.71e-03
 ooCMF Iter:    8 Total= -2119.187949609847 Active= -105.915731971279 G=     2.68e-03
 ooCMF Iter:    9 Total= -2119.187953013647 Active= -105.915735375078 G=     2.52e-03
 ooCMF Iter:   10 Total= -2119.187956758029 Active= -105.915739119460 G=     2.14e-03
 ooCMF Iter:   11 Total= -2119.187958485893 Active= -105.915740847324 G=     1.45e-03
 ooCMF Iter:   12 Total= -2119.187959072578 Active= -1

 * Status: success

 * Candidate solution
    Final objective value:     -2.119188e+03

 * Found with
    Algorithm:     Conjugate Gradient

 * Convergence measures
    |x - x'|               = 1.05e-06 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.42e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 9.55e-12 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 4.51e-15 ≰ 0.0e+00
    |g(x)|                 = 8.61e-07 ≤ 1.0e-06

 * Work counters
    Seconds run:   204  (vs limit Inf)
    Iterations:    27
    f(x) calls:    55
    ∇f(x) calls:   28


 ooCMF Iter:   28 Total= -2119.187959958186 Active= -105.915742319617 G=     3.33e-06
*ooCMF -2119.18795996 


┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /Users/nmayhall/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


In [8]:
using JLD2
# rotate integrals to new cmf optimized basis
ints = FermiCG.orbital_rotation(ints_original,U);

@save "cr_qubit_cmf_5clusters.jld2" ints clusters init_fspace da1 db1


Da = da1
Db = db1 
max_roots = 20

#
# form Cluster data
@time cluster_bases = FermiCG.compute_cluster_eigenbasis(ints, clusters, verbose=0,
                                                   max_roots=max_roots,
                                                   init_fspace=init_fspace,
                                                   rdm1a=Da, rdm1b=Db)


clustered_ham = FermiCG.extract_ClusteredTerms(ints, clusters)

cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);

FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, Da, Db);





896.486709 seconds (190.23 M allocations: 271.973 GiB, 8.33% gc time)
 Remove duplicates
 Number of terms reduced from  2525 to  1105


In [9]:
@save "cr_qubit_cmf_5clusters_bases_M20.jld2" ints clusters init_fspace da1 db1 cluster_bases


In [22]:
# Create BST state

v = FermiCG.BSTstate(clusters, FockConfig(init_fspace), cluster_bases, R=3)

FermiCG.add_single_excitons!(v, FockConfig(init_fspace), cluster_bases)
FermiCG.randomize!(v)
FermiCG.orthonormalize!(v)
e_ci, v = FermiCG.tucker_ci_solve(v, cluster_ops, clustered_ham);


UnitRange{Int64}[1:1, 1:1, 1:1, 1:1, 1:1]
TuckerConfig{5}((1:1, 1:1, 1:1, 1:1, 1:1))
 |== BST CI ========================================================
 Solve CI with # variables:                                96
 Cache Hamiltonian:                                  0.370623 seconds (41.12 k allocations: 186.953 MiB, 89.06% gc time)
 Iter:   1 SS: 3    E: -105.65971713  -105.62434298  -105.59939724   R: 1.0e-01  1.1e-01  9.9e-02   LinDep: 1.0e+00* 
 Iter:   2 SS: 6    E: -105.75963815  -105.74028422  -105.72556892   R: 8.5e-02  6.5e-02  7.0e-02   LinDep: 1.0e+00* 
 Iter:   3 SS: 9    E: -105.85342743  -105.77683271  -105.75528028   R: 9.5e-02  5.7e-02  3.4e-02   LinDep: 1.0e+00* 
 Iter:   4 SS: 12   E: -105.90822580  -105.81949416  -105.77002737   R: 4.2e-02  5.0e-02  4.0e-02   LinDep: 1.0e+00* 
 Iter:   5 SS: 15   E: -105.91454849  -105.83309294  -105.78937050   R: 1.5e-02  2.4e-02  3.9e-02   LinDep: 1.0e+00* 
 Iter:   6 SS: 18   E: -105.91558439  -105.83706064  -105.81005213   R: 6

In [23]:
using Printf
display(v, root=2)
for ei in 1:length(e_ci)
    @printf(" %5i %12.8f %12.8f eV\n", ei, e_ci[ei]+ints.h0, (e_ci[ei]-e_ci[1])*27.21165)
end


 --------------------------------------------------
 ---------- # Fockspaces -------------------:     1  
 ---------- # Configs    -------------------:    96  
 ---------- Root ---------------------------:     2  
 --------------------------------------------------
 Printing contributions greater than: 0.010000
 Weight              # configs (full)    (α,β)...            
 -------             --------- --------- ----------          
 1.000                                   ( 5,5 )( 3,3 )( 3,3 )( 3,3 )( 3,3 )
     1.000           19        19           2:20    1:1    1:1    1:1    1:1
     0.011           19        19            1:1    1:1    1:1   2:20    1:1
 ---------                               ----------          
 --------------------------------------------------

     1 -2119.18795996   0.00000000 eV
     2 -2119.11173022   2.07433692 eV
     3 -2119.11018598   2.11635829 eV


In [24]:
 e_var, v_var = FermiCG.block_sparse_tucker(v, cluster_ops, clustered_ham,
                                        max_iter    = 20,
                                        max_iter_pt = 200,
                                        nbody       = 4,
                                        H0          = "Hcmf",
                                        thresh_var  = 1e-1,
                                        thresh_foi  = 1e-3,
                                        thresh_pt   = 1e-3,
                                        ci_conv     = 1e-5,
                                        do_pt       = true,                          
                                        resolve_ss  = false,
                                        tol_tucker  = 1e-4);


 max_iter       : 20
 max_iter_pt    : 200
 nbody          : 4
 H0             : Hcmf
 thresh_var     : 0.1
 thresh_foi     : 0.001
 thresh_pt      : 0.001
 ci_conv        : 1.0e-5
 ci_max_iter    : 50
 ci_max_ss_vecs : 12
 resolve_ss     : false
 do_pt          : true
 tol_tucker     : 0.0001



     BST Iteration:    1 epsilon:   0.10000000
 Ref state compressed from:                                96 → 3          (thresh =  1.0e-01)
 Compute zeroth-order energy:                        0.005326 seconds (26.87 k allocations: 3.821 MiB)
 Compute <S^2>:                                      0.001475 seconds (8.29 k allocations: 2.374 MiB)
  Root       Energy           S2
     1 -105.91574232   2.00000000
     2 -105.83948225   0.00001945
     3 -105.83791596   0.00009158

 Compute FOIS. Reference space dim:                         3
 Number of tasks:                                         661
 Compute tasks:                                      0.267869 seconds (1.74 M allocations: 502.

In [28]:
@time e2 = FermiCG.compute_pt2_energy(v_var, cluster_ops, clustered_ham, thresh_foi=1e-5)
@time e2 = FermiCG.compute_pt2_energy(v_var, cluster_ops, clustered_ham, thresh_foi=1e-6)
@time e2 = FermiCG.compute_pt2_energy(v_var, cluster_ops, clustered_ham, thresh_foi=1e-7)

 |== Compute PT2 Energy ============================================
 H0          : Hcmf
 max_iter    : 50
 nbody       : 4
 thresh_foi  : 1.0e-5
 max_number  : nothing
 tol         : 1.0e-5
 opt_ref     : true
 verbose     : true

 Length of Reference:                                     843
 Solve zeroth-order problem:                       
 |== BST CI ========================================================
 Solve CI with # variables:                               843
 Cache Hamiltonian:                                  2.041514 seconds (7.80 M allocations: 1.142 GiB, 38.89% gc time)
 Iter:   1 SS: 3    E: -105.92845369* -105.85256989* -105.85232903*  R: 4.6e-06* 9.5e-06* 8.6e-06*  LinDep: 1.0e+00* 
 Diagonalization time:                               0.920534 seconds
 Compute <S^2>:                                      0.080045 seconds (1.05 M allocations: 72.991 MiB)
  Root       Energy           S2
     1 -105.92845369   2.00105974
     2 -105.85256989   0.00387920
     3 -105.8

3-element Vector{Float64}:
 -0.0010994161355029064
 -0.001214467927255214
 -0.001366939671925138

In [14]:
v1 = deepcopy(v_var);
display(v1,root=3)


 --------------------------------------------------
 ---------- # Fockspaces -------------------:    75  
 ---------- # Configs    -------------------:   843  
 ---------- Root ---------------------------:     3  
 --------------------------------------------------
 Printing contributions greater than: 0.010000
 Weight              # configs (full)    (α,β)...            
 -------             --------- --------- ----------          
 0.972                                   ( 5,5 )( 3,3 )( 3,3 )( 3,3 )( 3,3 )
     0.971           2         19           2:20    1:1    1:1    1:1    1:1
     0.012           1         19            1:1    1:1    1:1    1:1   2:20
     0.018           1         19            1:1    1:1   2:20    1:1    1:1
     0.012           1         19            1:1   2:20    1:1    1:1    1:1
     0.011           8         6859         2:20   2:20    1:1    1:1   2:20
     0.021           1         19            1:1    1:1    1:1   2:20    1:1
     0.010           8 

In [26]:
 e2, v2 = FermiCG.block_sparse_tucker(v1, cluster_ops, clustered_ham,
                                        max_iter    = 20,
                                        max_iter_pt = 200,
                                        nbody       = 4,
                                        H0          = "Hcmf",
                                        thresh_var  = 1e-1,
                                        thresh_foi  = 1e-4,
                                        thresh_pt   = 1e-3,
                                        ci_conv     = 1e-5,
                                        do_pt       = true,                          
                                        resolve_ss  = false,
                                        tol_tucker  = 1e-4);

 max_iter       : 20
 max_iter_pt    : 200
 nbody          : 4
 H0             : Hcmf
 thresh_var     : 0.1
 thresh_foi     : 0.0001
 thresh_pt      : 0.001
 ci_conv        : 1.0e-5
 ci_max_iter    : 50
 ci_max_ss_vecs : 12
 resolve_ss     : false
 do_pt          : true
 tol_tucker     : 0.0001



     BST Iteration:    1 epsilon:   0.10000000
 Ref state compressed from:                               301 → 3          (thresh =  1.0e-01)
 Compute zeroth-order energy:                        0.004223 seconds (28.87 k allocations: 3.636 MiB)
 Compute <S^2>:                                      0.001294 seconds (8.33 k allocations: 2.365 MiB)
  Root       Energy           S2
     1 -105.91574232   2.00000000
     2 -105.83937644   0.00000310
     3 -105.83802216   0.00008090

 Compute FOIS. Reference space dim:                         3
 Number of tasks:                                         661
 Compute tasks:                                      0.242109 seconds (2.02 M allocations: 395

In [28]:
display(v2, root=1, thresh=.1)



 --------------------------------------------------
 ---------- # Fockspaces -------------------:    75  
 ---------- # Configs    -------------------:   577  
 ---------- Root ---------------------------:     1  
 --------------------------------------------------
 Printing contributions greater than: 0.100000
 Weight              # configs (full)    (α,β)...            
 -------             --------- --------- ----------          
 0.984                                   ( 5,5 )( 3,3 )( 3,3 )( 3,3 )( 3,3 )
     0.984           1         1             1:1    1:1    1:1    1:1    1:1
 ---------                               ----------          
 --------------------------------------------------



In [29]:
 e3, v3 = FermiCG.block_sparse_tucker(v2, cluster_ops, clustered_ham,
                                        max_iter    = 20,
                                        max_iter_pt = 200,
                                        nbody       = 4,
                                        H0          = "Hcmf",
                                        thresh_var  = 1e-1,
                                        thresh_foi  = 1e-5,
                                        thresh_pt   = 1e-4,
                                        ci_conv     = 1e-5,
                                        do_pt       = true,                          
                                        resolve_ss  = false,
                                        tol_tucker  = 1e-4);

 max_iter       : 20
 max_iter_pt    : 200
 nbody          : 4
 H0             : Hcmf
 thresh_var     : 0.1
 thresh_foi     : 1.0e-5
 thresh_pt      : 0.0001
 ci_conv        : 1.0e-5
 ci_max_iter    : 50
 ci_max_ss_vecs : 12
 resolve_ss     : false
 do_pt          : true
 tol_tucker     : 0.0001



     BST Iteration:    1 epsilon:   0.10000000
 Ref state compressed from:                               577 → 3          (thresh =  1.0e-01)
 Compute zeroth-order energy:                        0.004680 seconds (26.48 k allocations: 3.526 MiB)
 Compute <S^2>:                                      0.001499 seconds (8.29 k allocations: 2.363 MiB)
  Root       Energy           S2
     1 -105.91574232   2.00000000
     2 -105.83895836   0.00002272
     3 -105.83844024   0.00006128

 Compute FOIS. Reference space dim:                         3
 Number of tasks:                                         661
 Compute tasks:                                      0.321741 seconds (2.18 M allocations: 55

In [30]:
# rotate integrals to new cmf optimized basis
ints = FermiCG.orbital_rotation(ints_original,U);

Da = da1
Db = db1 
max_roots = 50

#
# form Cluster data
cluster_bases = FermiCG.compute_cluster_eigenbasis(ints, clusters, verbose=1,
                                                   max_roots=max_roots,
                                                   init_fspace=init_fspace,
                                                   rdm1a=Da, rdm1b=Db,
                                                    delta_elec=2)

clustered_ham = FermiCG.extract_ClusteredTerms(ints, clusters)

cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);

FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, Da, Db);


IDX001:DIM1048576:001|002|003|004|005|006|007|008|009|010|
 Number of embedded electrons a,b: 12.000000 12.000000 FCIProblem:: #Orbs = 10  #α = 3  #β = 5  Dimension: 30240    
   State    1 Energy: -19.75503440 -2033.02725203
   State    2 Energy: -19.75402987 -2033.02624751
   State    3 Energy: -19.75204552 -2033.02426316
   State    4 Energy: -19.74869888 -2033.02091652
   State    5 Energy: -19.74699509 -2033.01921273
   State    6 Energy: -19.74461683 -2033.01683447
   State    7 Energy: -19.74290199 -2033.01511963
   State    8 Energy: -19.73946415 -2033.01168178
   State    9 Energy: -19.73782459 -2033.01004223
   State   10 Energy: -19.73513163 -2033.00734927
   State   11 Energy: -19.73253082 -2033.00474846
   State   12 Energy: -19.72784136 -2033.00005900
   State   13 Energy: -19.72348788 -2032.99570552
   State   14 Energy: -19.72323010 -2032.99544774
   State   15 Energy: -19.72207881 -2032.99429645
   State   16 Energy: -19.72047759 -2032.99269522
   State   17 Energy: -1

In [31]:
# Create BST state

v = FermiCG.BSTstate(clusters, FockConfig(init_fspace), cluster_bases, R=3)

FermiCG.add_single_excitons!(v, FockConfig(init_fspace), cluster_bases)
FermiCG.randomize!(v)
FermiCG.orthonormalize!(v)
e_ci, v = FermiCG.tucker_ci_solve(v, cluster_ops, clustered_ham);


UnitRange{Int64}[1:1, 1:1, 1:1, 1:1, 1:1]
TuckerConfig{5}((1:1, 1:1, 1:1, 1:1, 1:1))
 |== BST CI ========================================================
 Solve CI with # variables:                               246
 Cache Hamiltonian:                                  0.346969 seconds (54.82 k allocations: 198.850 MiB, 53.35% gc time)
 Iter:   1 SS: 3    E: -105.53570903  -105.51602273  -105.47799803   R: 1.7e-01  1.6e-01  1.7e-01   LinDep: 1.0e+00* 
 Iter:   2 SS: 6    E: -105.72430993  -105.69433586  -105.67775107   R: 1.2e-01  8.5e-02  9.5e-02   LinDep: 1.0e+00* 
 Iter:   3 SS: 9    E: -105.80672443  -105.72942556  -105.72232499   R: 1.2e-01  5.5e-02  5.7e-02   LinDep: 1.0e+00* 
 Iter:   4 SS: 12   E: -105.87849397  -105.75126951  -105.74513612   R: 8.0e-02  4.7e-02  5.2e-02   LinDep: 1.0e+00* 
 Iter:   5 SS: 15   E: -105.90257261  -105.76641682  -105.76203532   R: 4.6e-02  4.7e-02  3.9e-02   LinDep: 1.0e+00* 
 Iter:   6 SS: 18   E: -105.91079535  -105.78921195  -105.77016061   R: 2

In [32]:
 e3_50, v3_50 = FermiCG.block_sparse_tucker(v, cluster_ops, clustered_ham,
                                        max_iter    = 20,
                                        max_iter_pt = 200,
                                        nbody       = 4,
                                        H0          = "Hcmf",
                                        thresh_var  = 1e-1,
                                        thresh_foi  = 1e-5,
                                        thresh_pt   = 1e-4,
                                        ci_conv     = 1e-5,
                                        do_pt       = true,                          
                                        resolve_ss  = false,
                                        tol_tucker  = 1e-4);

 max_iter       : 20
 max_iter_pt    : 200
 nbody          : 4
 H0             : Hcmf
 thresh_var     : 0.1
 thresh_foi     : 1.0e-5
 thresh_pt      : 0.0001
 ci_conv        : 1.0e-5
 ci_max_iter    : 50
 ci_max_ss_vecs : 12
 resolve_ss     : false
 do_pt          : true
 tol_tucker     : 0.0001



     BST Iteration:    1 epsilon:   0.10000000
 Ref state compressed from:                               246 → 3          (thresh =  1.0e-01)
 Compute zeroth-order energy:                        0.020636 seconds (28.84 k allocations: 5.684 MiB)
 Compute <S^2>:                                      0.001801 seconds (8.29 k allocations: 2.383 MiB)
  Root       Energy           S2
     1 -105.91574232   2.00000000
     2 -105.83948221   0.00002033
     3 -105.83791584   0.00009700

 Compute FOIS. Reference space dim:                         3
 Number of tasks:                                         661
 Compute tasks:                                      2.572102 seconds (2.26 M allocations: 2.

In [ ]:
# rotate integrals to new cmf optimized basis
ints = FermiCG.orbital_rotation(ints_original,U);

Da = da1
Db = db1 
max_roots = 100

#
# form Cluster data
@time cluster_bases = FermiCG.compute_cluster_eigenbasis(ints, clusters, verbose=0,
                                                   max_roots=max_roots,
                                                   init_fspace=init_fspace,
                                                   rdm1a=Da, rdm1b=Db)

clustered_ham = FermiCG.extract_ClusteredTerms(ints, clusters)

@time cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);

FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, Da, Db);


In [37]:
# Create BST state

v = FermiCG.BSTstate(clusters, FockConfig(init_fspace), cluster_bases, R=3)

FermiCG.add_single_excitons!(v, FockConfig(init_fspace), cluster_bases)
FermiCG.randomize!(v)
FermiCG.orthonormalize!(v)
e_ci, v = FermiCG.tucker_ci_solve(v, cluster_ops, clustered_ham);


UnitRange{Int64}[1:1, 1:1, 1:1, 1:1, 1:1]
TuckerConfig{5}((1:1, 1:1, 1:1, 1:1, 1:1))
 |== BST CI ========================================================
 Solve CI with # variables:                               246
 Cache Hamiltonian:                                  0.536894 seconds (54.44 k allocations: 198.826 MiB, 58.47% gc time)
 Iter:   1 SS: 3    E: -105.51984639  -105.50552956  -105.49467108   R: 1.7e-01  1.7e-01  1.7e-01   LinDep: 1.0e+00* 
 Iter:   2 SS: 6    E: -105.70946850  -105.69401651  -105.67603194   R: 9.4e-02  9.9e-02  9.4e-02   LinDep: 1.0e+00* 
 Iter:   3 SS: 9    E: -105.75456406  -105.74409973  -105.72286383   R: 7.9e-02  7.3e-02  6.6e-02   LinDep: 1.0e+00* 
 Iter:   4 SS: 12   E: -105.82688120  -105.77930841  -105.75256249   R: 1.1e-01  6.6e-02  6.1e-02   LinDep: 1.0e+00* 
 Iter:   5 SS: 15   E: -105.88790202  -105.80751698  -105.78369174   R: 7.7e-02  5.8e-02  7.1e-02   LinDep: 1.0e+00* 
 Iter:   6 SS: 18   E: -105.90981268  -105.82596852  -105.81522369   R: 3

In [38]:
 e3_50, v3_50 = FermiCG.block_sparse_tucker(v, cluster_ops, clustered_ham,
                                        max_iter    = 20,
                                        max_iter_pt = 200,
                                        nbody       = 4,
                                        H0          = "Hcmf",
                                        thresh_var  = 1e-1,
                                        thresh_foi  = 1e-5,
                                        thresh_pt   = 1e-4,
                                        ci_conv     = 1e-5,
                                        do_pt       = true,                          
                                        resolve_ss  = false,
                                        tol_tucker  = 1e-4);

 max_iter       : 20
 max_iter_pt    : 200
 nbody          : 4
 H0             : Hcmf
 thresh_var     : 0.1
 thresh_foi     : 1.0e-5
 thresh_pt      : 0.0001
 ci_conv        : 1.0e-5
 ci_max_iter    : 50
 ci_max_ss_vecs : 12
 resolve_ss     : false
 do_pt          : true
 tol_tucker     : 0.0001



     BST Iteration:    1 epsilon:   0.10000000
 Ref state compressed from:                               246 → 3          (thresh =  1.0e-01)
 Compute zeroth-order energy:                        0.014128 seconds (28.82 k allocations: 5.682 MiB)
 Compute <S^2>:                                      0.001365 seconds (8.29 k allocations: 2.391 MiB)
  Root       Energy           S2
     1 -105.91574232   2.00000000
     2 -105.83948221   0.00002032
     3 -105.83791583   0.00009758

 Compute FOIS. Reference space dim:                         3
 Number of tasks:                                         661
 Compute tasks:                                      2.314394 seconds (2.26 M allocations: 2.